# 🌐 LangGraph Agents with Llama Stack: Bring Your Own Agentic Framework

This notebook demonstrates **agentic framework integration** - how to use any agentic framework (LangGraph, AutoGen, CrewAI) with Llama Stack's OpenAI-compatible APIs.

**What is LangGraph?**
LangGraph is a state-based agent framework that transforms AI applications into sophisticated multi-step reasoning systems:
- **State Management**: Track conversation history and intermediate reasoning steps
- **Graph-Based Flows**: Define complex agent workflows with conditional logic and loops
- **Tool Integration**: Seamlessly bind external tools for enhanced capabilities
- **Flexible Architecture**: Build everything from simple chatbots to complex multi-agent systems

**Why LangGraph + Llama Stack?**
Instead of being locked into a single provider's ecosystem, this combination gives you:
- **Framework Freedom**: Use your preferred agentic framework without vendor lock-in
- **OpenAI Compatibility**: Leverage existing LangChain/LangGraph code with minimal changes
- **Tool Ecosystem**: Access MCP (Model Context Protocol) tools for weather, web search, and more
- **Production Ready**: Deploy on your infrastructure with full observability and control

## 🏗️ LangGraph + Llama Stack Architecture

The integration creates **three key layers** that work together to enable sophisticated agentic capabilities:

### 1. 🧠 LangGraph Layer (The Agent Brain)
This is where intelligent agent behavior is defined:
- **StateGraph**: Manages conversation state and agent memory across interactions
- **Nodes & Edges**: Define agent reasoning steps and decision flow
- **Message Handling**: Tracks conversation history and context
- **Conditional Logic**: Enables complex multi-step reasoning workflows

### 2. 🔗 OpenAI Compatibility Layer (The Translation)
This bridges LangGraph to Llama Stack seamlessly:
- **ChatOpenAI Client**: Standard LangChain interface pointing to Llama Stack
- **OpenAI-Compatible Endpoint**: Llama Stack's `/v1/openai/v1` endpoint
- **Tool Binding**: Attach MCP tools to LLM for enhanced capabilities
- **Response Handling**: Process streaming and non-streaming responses

### 3. 🦙 Llama Stack Layer (The AI API to run them all)
This provides the AI model and tool runtime:
- **Model Inference**: vLLM-powered Llama 3.2 3B for fast, local inference
- **MCP Tools**: Weather, web search, and custom tool integrations
- **Observability**: Comprehensive telemetry and monitoring
- **Production Features**: Safety filters, rate limiting, and error handling

**The Power**: LangGraph provides sophisticated agent orchestration while Llama Stack handles the heavy lifting of model inference and tool execution.

## 📦 Install Required Packages

Install the LangGraph and integration dependencies:

In [6]:
!pip install -q langgraph==0.6.7 langchain-openai==0.3.32 langchain-core==0.3.75


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
# Core imports for LangGraph integration
import os
import sys
import json
from typing import Annotated
from typing_extensions import TypedDict

# LangGraph imports for agent creation
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages

# LangChain imports for OpenAI compatibility
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool

# Additional utilities for development
from pprint import pprint

## 🔗 Connect LangGraph to Llama Stack

Connect LangGraph to Llama Stack's OpenAI-compatible endpoint. This creates a seamless bridge that allows LangGraph to use Llama Stack as its inference backend while maintaining full compatibility with existing LangChain code.

**Key Integration Points:**
- **OpenAI-Compatible Endpoint**: Use Llama Stack's `/v1/openai/v1` endpoint for seamless integration
- **Model Configuration**: Point to deployed Llama 3.2 3B model for fast inference
- **Tool Binding**: Prepare for MCP weather tool integration
- **State Management**: Set up LangGraph's conversation state handling

In [8]:
# === LangGraph + Llama Stack Configuration ===
print("🌐 Configuring LangGraph with Llama Stack Integration")

# === Endpoint Configuration ===
# Use Llama Stack's OpenAI-compatible endpoint for seamless LangChain integration
LLAMA_STACK_OPENAI_ENDPOINT = "http://llama-stack-instance-service.llama-serve.svc.cluster.local:8321/v1/openai/v1"
INFERENCE_MODEL = "llama3-2-3b"  # Model deployed in the cluster
API_KEY = "fake"       # Not needed for local deployment

print(f"📍 Llama Stack OpenAI Endpoint: {LLAMA_STACK_OPENAI_ENDPOINT}")
print(f"🤖 Inference Model: {INFERENCE_MODEL}")

# === Create ChatOpenAI Client ===
# This creates a standard LangChain client that talks to Llama Stack
llm = ChatOpenAI(
    model=INFERENCE_MODEL,
    openai_api_key=API_KEY,
    openai_api_base=LLAMA_STACK_OPENAI_ENDPOINT,
    use_responses_api=True,
)

print("✅ ChatOpenAI client configured for Llama Stack")

# === Test Basic Connectivity ===
print("\n🧪 Testing basic LangGraph-Llama Stack connectivity...")

try:
    # Simple connectivity test
    response = llm.invoke("Hello! Please respond with 'Connection successful' if you can hear me.")
    print(f"📤 Test Query: Hello connectivity test")
    print(f"📥 Response: {response.content}")
    print("✅ Connection successful!")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("💡 Make sure Llama Stack service is running and accessible")
    sys.exit(1)

print(f"\n🎯 LangGraph is now ready to use Llama Stack for inference!")

🌐 Configuring LangGraph with Llama Stack Integration
📍 Llama Stack OpenAI Endpoint: http://llama-stack-instance-service.llama-serve.svc.cluster.local:8321/v1/openai/v1
🤖 Inference Model: llama3-2-3b
✅ ChatOpenAI client configured for Llama Stack

🧪 Testing basic LangGraph-Llama Stack connectivity...
📤 Test Query: Hello connectivity test
📥 Response: [{'type': 'text', 'text': 'Connection successful', 'annotations': []}]
✅ Connection successful!

🎯 LangGraph is now ready to use Llama Stack for inference!


## 🤖 Step 1: Create a Basic LangGraph Agent

Now let's build our first LangGraph agent that uses Llama Stack for inference. This demonstrates the core pattern for creating stateful, conversational agents.

**LangGraph Fundamentals:**
1. **State Definition**: Define what information the agent tracks across interactions
2. **Node Creation**: Create functions that process messages and update state
3. **Graph Building**: Connect nodes with edges to define conversation flow
4. **Compilation**: Compile the graph into an executable agent

**Key Benefits of LangGraph:**
- **Memory**: Automatically tracks conversation history across interactions
- **State Persistence**: Maintains context and intermediate results
- **Flexible Flow**: Define complex multi-step reasoning patterns
- **Tool Integration**: Easily bind external tools for enhanced capabilities

In [9]:
print("📋 LangGraph + Llama Stack Integration")
print(f"   LLAMA_STACK_URL: {LLAMA_STACK_OPENAI_ENDPOINT}")
print(f"   INFERENCE_MODEL: {INFERENCE_MODEL}")

llm = ChatOpenAI(
    model=INFERENCE_MODEL,
    openai_api_key=API_KEY,
    openai_api_base=LLAMA_STACK_OPENAI_ENDPOINT,
    use_responses_api=True,
)

# Test connectivity
print("\n🧪 Testing basic connectivity:")
response = llm.invoke("Hello")
print(f"✅ Connection successful")

# Bind MCP weather tools
print("\n🛠️ Setting up MCP weather tools...")
llm_with_tools = llm.bind_tools([
    {
        "type": "mcp",
        "server_label": "weather",
        "server_url": "http://mcp-weather.llama-serve.svc.cluster.local:80/sse",
        "require_approval": "never",
    },
])
print("✅ MCP tools configured")

# Define LangGraph State and Agent
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    return {"messages": [message]}

# Build LangGraph StateGraph
print("\n🏗️ Building LangGraph agent...")
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()
print("✅ LangGraph agent ready")

# Test the integration
print("\n" + "="*50)
print("🚀 Testing LangGraph Agent with MCP Tools")
print("="*50)

response = graph.invoke({
    "messages": [{"role": "user", "content": "What is the weather in Boston?"}]
})

print("Weather Response:")
for message in response['messages']:
    if hasattr(message, 'content'):
        if isinstance(message.content, list):
            for content_block in message.content:
                if content_block.get('type') == 'text':
                    print(content_block.get('text', ''))
        elif isinstance(message.content, str):
            print(message.content)
    else:
        message.pretty_print()

📋 LangGraph + Llama Stack Integration
   LLAMA_STACK_URL: http://llama-stack-instance-service.llama-serve.svc.cluster.local:8321/v1/openai/v1
   INFERENCE_MODEL: llama3-2-3b

🧪 Testing basic connectivity:


✅ Connection successful

🛠️ Setting up MCP weather tools...
✅ MCP tools configured

🏗️ Building LangGraph agent...
✅ LangGraph agent ready

🚀 Testing LangGraph Agent with MCP Tools
Weather Response:
What is the weather in Boston?
Unfortunately, I don't have direct access to real-time weather information. However, I can suggest some alternative ways for you to find the current weather in Boston. You can check online weather websites such as weather.com or accuweather.com, or download a weather app on your smartphone to get the latest forecast.
